In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier 
import numpy as np
from keras.datasets import mnist
from modAL.models import ActiveLearner

Using TensorFlow backend.


In [2]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    
    return model

In [3]:
clf = KerasClassifier(create_model)

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32') /255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

n_initial = 1000
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)
X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

X_pool = np.delete(X_train, initial_idx, axis=0)[:5000]
y_pool = np.delete(y_train, initial_idx, axis=0)[:5000]

In [5]:
learner = ActiveLearner(estimator=clf,
                       X_training=X_initial,
                       y_training=y_initial,
                       verbose=1)

Epoch 1/1
1000/1000 [==============================] - 3s 3ms/step - loss: 1.5128 - accuracy: 0.4790


In [6]:
n_queries = 10
for idx in range(n_queries):
    print('Query no. %d' %(idx+1))
    query_idx, query_instance = learner.query(X_pool, n_instances=100, verbose=0)
    learner.teach(X=X_pool[query_idx], y=y_pool[query_idx], only_new=False, verbose=1)
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)

Query no. 1
Epoch 1/1
1100/1100 [==============================] - 4s 3ms/step - loss: 1.5313 - accuracy: 0.4809
Query no. 2
Epoch 1/1
1200/1200 [==============================] - 4s 3ms/step - loss: 1.6054 - accuracy: 0.4650
Query no. 3
Epoch 1/1
1300/1300 [==============================] - 4s 3ms/step - loss: 1.4716 - accuracy: 0.4869
Query no. 4
Epoch 1/1
1400/1400 [==============================] - 4s 3ms/step - loss: 1.4794 - accuracy: 0.5000
Query no. 5
Epoch 1/1
1500/1500 [==============================] - 5s 3ms/step - loss: 1.4783 - accuracy: 0.5013
Query no. 6
Epoch 1/1
1600/1600 [==============================] - 5s 3ms/step - loss: 1.5443 - accuracy: 0.4631
Query no. 7
Epoch 1/1
1700/1700 [==============================] - 5s 3ms/step - loss: 1.4287 - accuracy: 0.5224
Query no. 8
Epoch 1/1
1800/1800 [==============================] - 6s 3ms/step - loss: 1.4864 - accuracy: 0.4894
Query no. 9
Epoch 1/1
1900/1900 [==============================] - 6s 3ms/step - loss: 1.4487 - 